In [4]:
from eos.relativistic_ISCT import Relativistic_ISCT
import os
import pandas as pd

def get_parameters_from_filename(filename):

    
    def get_values(array, string):
        for val in array:
            new_str = string + str(val)
            if new_str in filename:
                return val

    if 'baryons' in filename:
        m = 940.
    elif 'pions' in filename:
        m = 140.
    else:
        m = get_values([20., 25., 30.], 'm_')

    R = get_values([0.39, 0.4], 'R_')
    label = get_values(['ISCT2', 'IST', 'vdw', 'ISCT'], '')
    b = get_values([1., 0.], 'b_')
    g = get_values([3., 30., 300., 3000., 4., 40., 400., 4000.], 'g_')
    mu = get_values([0., 200., 400., 600., 1000., 2000.], 'mu_')

    return label, m, b, R, g, mu

In [7]:
foldername = 'cs_sq_fulldata'
for file in os.listdir(foldername):
    if os.path.isfile(os.path.join(foldername, file)):
        label, m, b, R, g, mu = get_parameters_from_filename(file)
        if b==1.:
            print('*'*30)
            print(file,'\n')
            
            df = pd.read_csv(os.path.join(foldername, file))
            eos = Relativistic_ISCT(eos=label, m=m, g=g, R=R, b=b)
            row = df.loc[df['T'] == 3000.]
            try:
                T, p, Sigma, K = row[['T', 'p', 'Sigma', 'K']].to_numpy()[0]
            except:
                continue
            vratio = eos.v_eff_ratio(0, T, p, Sigma, K)
            print(f'T:\t {T}\t\t v ratio :\t {vratio}')
            print('*'*30)

******************************
cs_sq_extradata_baryons_b_1.0_ISCT2_g_400.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.2841917]
******************************
******************************
cs_sq_extradata_baryons_b_1.0_ISCT2_g_4000.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.27294643]
******************************
******************************
cs_sq_extradata_baryons_b_1.0_ISCT_g_400.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.29591348]
******************************
******************************
cs_sq_extradata_baryons_b_1.0_ISCT_g_4000.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.28850975]
******************************
******************************
cs_sq_extradata_pions_b_1.0_ISCT2_g_300.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.24004543]
******************************
******************************
cs_sq_extradata_pions_b_1.0_ISCT2_g_3000.0_R_0.39_mu_0.0_.csv 

T:	 3000.0		 v ratio :	 [0.2324392]
******************************
*********************